# 헬스케어 멀티에이전트 챗봇 (OpenAI)

사용자의 질문에 따라 전문 에이전트에게 라우팅하는 챗봇입니다.

## 모델
- **GPT-4o-mini**: OpenAI의 경량 모델

## 구조
```
사용자 → Supervisor → 운동 에이전트 (exercise)
                    → 건강 에이전트 (health)
                    → 식단 에이전트 (diet)
                    → Synthesizer → 종료
```

## 특징
- **Checkpointer**: 대화 히스토리 유지
- **Supervisor**: 질문 유형에 따라 적절한 에이전트로 라우팅
- **agent_responses**: 각 에이전트 응답을 별도 저장
- **Synthesizer**: 모든 응답을 종합하여 최종 답변 생성

## 1. 환경 설정

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

# OpenAI API Key 확인
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
MODEL_NAME = "gpt-4o-mini"

if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY가 설정되지 않았습니다. .env 파일을 확인하세요.")

print(f"Model: {MODEL_NAME}")
print("OpenAI API Key: " + "*" * 20 + OPENAI_API_KEY[-4:])

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model=MODEL_NAME,
    temperature=0.7,
)

print("LLM 초기화 완료 (GPT-4o-mini)")

## 2. 상태 및 에이전트 정의

In [ ]:
from typing import Literal, Annotated
from pydantic import BaseModel, Field
from operator import add

from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.types import Command
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage


class State(MessagesState):
    """대화 상태"""
    next: str = ""
    pending_agents: list[str] = Field(default_factory=list)
    completed_agents: list[str] = Field(default_factory=list)
    agent_responses: dict[str, str] = Field(default_factory=dict)  # 에이전트별 응답 저장


# 에이전트 목록
MEMBERS = ["exercise", "health", "diet"]

print("State 정의 완료")
print(f"에이전트: {MEMBERS}")
print("필드: pending_agents, completed_agents, agent_responses")

In [ ]:
# Supervisor 정의

# 질문 분석용 프롬프트 (필요한 에이전트 리스트 추출)
ANALYZER_PROMPT = """Analyze the user's question and determine which specialists are needed.

Specialists:
- exercise: 운동, 헬스, 트레이닝, 근육, 스트레칭 관련
- health: 건강, 질병, 수면, 스트레스 관련
- diet: 식단, 영양, 다이어트, 칼로리 관련

Return a list of needed specialists. Examples:
- "살빼려면?" → ["diet", "exercise"]
- "잠을 못자요" → ["health"]
- "운동하고 잘 먹고 싶어" → ["exercise", "diet"]
- "안녕" or "고마워" → []

Return ONLY the specialists that are clearly needed.
"""

class AgentList(BaseModel):
    """필요한 에이전트 리스트"""
    agents: list[str] = Field(description="List of needed agents: exercise, health, diet")


def supervisor_node(state: State) -> Command[Literal["exercise", "health", "diet", "synthesizer", "__end__"]]:
    """Supervisor: 질문 분석 및 라우팅"""
    
    pending = state.get("pending_agents", [])
    completed = state.get("completed_agents", [])
    
    print(f"\n[Supervisor] pending={pending}, completed={completed}")
    
    # 첫 호출: pending_agents가 비어있으면 질문 분석
    if not pending and not completed:
        print("[Supervisor] 질문 분석 중...")
        
        messages = [
            {"role": "system", "content": ANALYZER_PROMPT},
        ] + state["messages"]
        
        response = llm.with_structured_output(AgentList).invoke(messages)
        pending = response.agents
        
        print(f"[Supervisor] 필요한 에이전트: {pending}")
        
        if not pending:
            print("[Supervisor] → END (에이전트 필요 없음)")
            return Command(goto=END, update={"next": "FINISH", "pending_agents": [], "completed_agents": []})
    
    # pending에서 다음 에이전트 선택
    if pending:
        next_agent = pending[0]
        remaining = pending[1:]
        new_completed = completed + [next_agent]
        
        print(f"[Supervisor] → {next_agent}")
        
        return Command(
            goto=next_agent,
            update={
                "next": next_agent,
                "pending_agents": remaining,
                "completed_agents": new_completed
            }
        )
    
    # pending이 비면 → synthesizer로 이동 (최종 응답 생성)
    print(f"[Supervisor] → Synthesizer")
    return Command(goto="synthesizer", update={"next": "synthesizer"})

print("Supervisor 정의 완료")

In [ ]:
# 전문 에이전트들 정의

EXERCISE_PROMPT = """당신은 운동 전문가입니다.
사용자의 운동 관련 질문에 친절하고 전문적으로 답변하세요.
- 운동 방법, 루틴, 자세 교정
- 근력 운동, 유산소 운동 추천
- 스트레칭, 부상 예방

한국어로 간결하게 답변하세요 (3-5문장)."""

HEALTH_PROMPT = """당신은 건강 상담사입니다.
사용자의 건강 관련 질문에 친절하고 전문적으로 답변하세요.
- 일반적인 건강 정보, 생활 습관
- 수면, 스트레스 관리
- 증상에 대한 일반적인 조언

한국어로 간결하게 답변하세요 (3-5문장)."""

DIET_PROMPT = """당신은 영양사입니다.
사용자의 식단 관련 질문에 친절하고 전문적으로 답변하세요.
- 균형 잡힌 식단 추천
- 다이어트, 체중 관리
- 영양소, 음식 정보

한국어로 간결하게 답변하세요 (3-5문장)."""

SYNTHESIZER_PROMPT = """당신은 헬스케어 종합 상담사입니다.
아래 전문가들의 조언을 바탕으로 사용자에게 통합된 답변을 제공하세요.

## 전문가 조언
{agent_responses}

## 지침
1. 각 전문가의 핵심 내용을 종합하세요
2. 중복을 제거하고 일관성 있게 정리하세요
3. 친절하고 자연스러운 한국어로 답변하세요
"""


def make_agent_node(name: str, system_prompt: str):
    """에이전트 노드 생성 팩토리"""
    def agent_node(state: State) -> Command[Literal["supervisor"]]:
        pending = state.get("pending_agents", [])
        completed = state.get("completed_agents", [])
        current_responses = state.get("agent_responses", {})
        
        print(f"[{name}] 응답 생성 중...")
        
        # 사용자 질문만 가져와서 응답 생성
        messages = [
            SystemMessage(content=system_prompt),
        ] + state["messages"]
        
        response = llm.invoke(messages)
        
        # agent_responses에 저장 (messages에 추가하지 않음)
        new_responses = {**current_responses, name: response.content}
        
        print(f"[{name}] 완료 → supervisor")
        
        return Command(
            update={
                "agent_responses": new_responses,
                "pending_agents": pending,
                "completed_agents": completed,
            },
            goto="supervisor",
        )
    return agent_node


def synthesizer_node(state: State):
    """Synthesizer: agent_responses를 종합하여 최종 답변 생성"""
    responses = state.get("agent_responses", {})
    
    print(f"[Synthesizer] 종합 중... (에이전트: {list(responses.keys())})")
    
    # 에이전트 응답들을 포맷팅
    formatted_responses = "\n\n".join([
        f"### {agent} 전문가\n{content}" 
        for agent, content in responses.items()
    ])
    
    # Synthesizer 프롬프트에 에이전트 응답 삽입
    prompt = SYNTHESIZER_PROMPT.format(agent_responses=formatted_responses)
    
    messages = [
        SystemMessage(content=prompt),
    ] + state["messages"]  # 원본 사용자 질문 포함
    
    response = llm.invoke(messages)
    
    print("[Synthesizer] 완료 → END")
    
    # 최종 응답만 messages에 추가 (AIMessage)
    return {
        "messages": [AIMessage(content=response.content)],
        "next": "FINISH",
        "pending_agents": [],
        "completed_agents": [],
        "agent_responses": {},  # 초기화
    }


# 에이전트 노드 생성
exercise_node = make_agent_node("exercise", EXERCISE_PROMPT)
health_node = make_agent_node("health", HEALTH_PROMPT)
diet_node = make_agent_node("diet", DIET_PROMPT)

print("에이전트 정의 완료: exercise, health, diet, synthesizer")

## 3. 그래프 구성

In [ ]:
from langgraph.checkpoint.memory import MemorySaver

# Checkpointer - 대화 히스토리 유지
memory = MemorySaver()

# 그래프 구성
builder = StateGraph(State)

# 노드 추가
builder.add_node("supervisor", supervisor_node)
builder.add_node("exercise", exercise_node)
builder.add_node("health", health_node)
builder.add_node("diet", diet_node)
builder.add_node("synthesizer", synthesizer_node)

# 시작점
builder.add_edge(START, "supervisor")

# Synthesizer → END
builder.add_edge("synthesizer", END)

# 컴파일
chatbot = builder.compile(checkpointer=memory)

print("챗봇 그래프 컴파일 완료")
print("구조: START → supervisor → [agents] → supervisor → synthesizer → END")

In [ ]:
from IPython.display import Image, display

display(Image(chatbot.get_graph().draw_mermaid_png()))

## 4. 대화 함수

In [ ]:
def chat(message: str, thread_id: str = "user-1"):
    """챗봇과 대화"""
    config = {"configurable": {"thread_id": thread_id}}
    
    print(f"\n{'='*60}")
    print(f"[사용자] {message}")
    print(f"{'='*60}")
    
    for event in chatbot.stream(
        {"messages": [("user", message)]},
        config,
        stream_mode="updates"
    ):
        for node_name, node_output in event.items():
            # Synthesizer의 최종 응답만 표시
            if node_name == "synthesizer" and "messages" in node_output:
                for msg in node_output["messages"]:
                    if hasattr(msg, "content") and msg.content:
                        print(f"\n[답변]")
                        print(msg.content)


def show_history(thread_id: str = "user-1"):
    """대화 히스토리 확인"""
    config = {"configurable": {"thread_id": thread_id}}
    state = chatbot.get_state(config)
    
    print(f"\n{'='*60}")
    print(f"대화 히스토리 (thread: {thread_id})")
    print(f"{'='*60}")
    
    if state.values:
        for i, msg in enumerate(state.values.get("messages", [])):
            role = msg.type if hasattr(msg, 'type') else 'unknown'
            content = msg.content[:100] + "..." if len(msg.content) > 100 else msg.content
            print(f"[{i+1}] {role}: {content}")

print("대화 함수 정의 완료")

## 5. 테스트

In [ ]:
# 운동 관련 질문
chat("헬스장에서 할 수 있는 가슴 운동 추천해줘")

In [ ]:
# 식단 관련 질문
chat("다이어트 할 때 아침에 뭘 먹으면 좋아?")

In [ ]:
# 건강 관련 질문
chat("요즘 잠을 잘 못자는데 어떻게 하면 좋을까?")

In [ ]:
# 복합 질문 (여러 에이전트 필요)
chat("건강하게 살을 빼고 싶은데 어떻게 해야 해?", thread_id="user-2")

In [ ]:
# 대화 히스토리 확인
show_history(thread_id="user-2")

## 요약

### 구조
```
User Message
     ↓
Supervisor (질문 분석 → pending_agents 리스트 생성)
     ↓
┌────┼────┬────┐
↓    ↓    ↓    ↓
운동  건강  식단  (에이전트 없으면 바로 END)
 ↓    ↓    ↓
 └────┴────┘
      ↓
 agent_responses에 각 응답 저장
      ↓
Supervisor (pending 비면 → synthesizer)
      ↓
Synthesizer (agent_responses 종합 → 최종 AIMessage)
      ↓
     END
```

### 핵심 포인트
- **GPT-4o-mini**: OpenAI 경량 모델 사용
- **agent_responses**: 각 에이전트 응답을 dict로 저장 (messages에 추가 X)
- **Synthesizer**: agent_responses만 참조하여 종합 응답 생성
- **messages 최소화**: user 질문 + 최종 AI 응답만 저장
- **Checkpointer**: thread_id별로 대화 히스토리 유지